# Physics Informed Neural Network
## Solving 2D Steady Navier-Stokes Equations

In [2]:
import torch
import math
from points_generator_optimized import generator
from loss_function_steady_optimized import Loss
from model_optimized import PINN
import numpy as np
import matplotlib.pyplot as plt

### Geometrical parameters

In [ ]:
x_start = 0; x_end = 2.2   # x-length
y_start = 0; y_end = 0.41  # y-lenght
t_start = 0; t_end = 0    # t_lenght
xc = 0.4; yc = 0.2; rr = 0.05  # circle-coordinates
D = 2*rr
rho = 1000  # density
nu = 1e-4   # kinematic viscosity
U_max = 0.05 # current maximum velocity
n_interior = 5000  # interior points
n_boundary = 1000   # boundary points
x_domain = torch.tensor([x_start,x_end])
y_domain = torch.tensor([y_start,y_end])
t_domain = torch.tensor([t_start,t_end])
circle_coordinates = torch.tensor([xc,yc,rr])
distance = torch.tensor([2 * rr])  # distance from the center for extra sampling
properties = torch.tensor([rho,nu,U_max])
number_intervals = torch.tensor([10])   # time intervals
n_points = torch.tensor([n_interior,n_boundary])

### Network parameters

In [ ]:
device = 'cpu'  #'cuda' if torch.cuda.is_available() else 'cpu'
num_input = 2; num_output = 3; num_layers = 3; num_neurons = 256
sigma_x = 10; sigma_y = 10
mu = 0.5; std = 0.1
epsilon = 1
lambda_frequency = 500
alpha = 0.9

network_parameters = torch.tensor([num_input,num_output,num_layers,num_neurons])
fourier_features_parameters = torch.tensor([sigma_x,sigma_y])
random_weight_initialization_parameters = torch.tensor([mu,std])
other_hyperparameters = torch.tensor([epsilon,lambda_frequency,alpha])

### Initializing classes

In [ ]:
points_generator = generator(x_domain,
                             y_domain,
                             t_domain,
                             circle_coordinates,
                             distance,
                             properties,
                             number_intervals,
                             n_points)

In [ ]:
pinn = PINN(network_parameters,
          fourier_features_parameters,
          x_domain,
          y_domain,
          circle_coordinates,
          properties,
          random_weight_initialization_parameters,
          device)

In [ ]:
loss_fn = Loss(x_domain,
              y_domain,
              circle_coordinates,
              properties,
              n_points,
              other_hyperparameters,
              device)

### Chechk points

In [ ]:
x_res, y_res, _ = points_generator.generate_interior_points()
x_cir, y_cir, _ = points_generator.generate_circle_points()
down,up,left,right = points_generator.generate_boundary_points()

x_d = down[:,0]; y_d = down[:,1]
x_u = up[:,0]; y_u = up[:,1]
x_l = left[:,0]; y_l = left[:,1]
x_r = right[:,0]; y_r = right[:,1]

plt.figure(figsize=(20,10))
plt.scatter(x_res,y_res,c='b',s=2,label='Domain')
plt.scatter(x_cir,y_cir,c='r',s=2,label='Circle')
plt.scatter(x_l,y_l,c='g',s=7,label='Boundary')
plt.scatter(x_u,y_u,c='g',s=7,label='Boundary')
plt.scatter(x_d,y_d,c='g',s=7,label='Boundary')
plt.scatter(x_r,y_r,c='g',s=7,label='Boundary')
# Imposto limiti assi
plt.xlim(0, x_end/D)
plt.ylim(0, y_end/D)
# Imposto rapporto assi NON uguale
plt.gca().set_aspect(1)
plt.legend()
plt.show()
plt.show()

### Initialing optimizer

In [ ]:
optimizer = torch.optim.Adam(params=pinn.parameters(),lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,1000, gamma=0.8)
weights = torch.tensor([1,1,1,1,1,1,1,1])
MAX_EPOCHS = 200000

In [ ]:
TOTAL_loss = []
PDE_loss = []
BC_loss = []

### Plot check coordinates

In [ ]:
n_slices = 500
xx = torch.linspace(0,x_end/D,n_slices); yy = torch.linspace(0,y_end/D,n_slices)
x_grid, y_grid = torch.meshgrid(xx,yy,indexing='xy')
x_grid = x_grid.reshape(-1,1); y_grid = y_grid.reshape(-1,1)
mask_tensor = (x_grid*D - xc)**2 + (y_grid*D - yc)**2 >= rr**2
x_pinn = x_grid[mask_tensor].reshape(-1,1); y_pinn = y_grid[mask_tensor].reshape(-1,1)
U = torch.full_like(x_grid,float('nan'))
V = torch.full_like(x_grid,float('nan'))
P = torch.full_like(x_grid,float('nan'))

### Creating saving directory

In [ ]:
from pathlib import Path

# 1. Create models directory 
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path 
MODEL_NAME = "2D_steady_model_optimized.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict 
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=pinn.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH) 

In [ ]:
for epoch in range(MAX_EPOCHS):

    pinn.train()
    optimizer.zero_grad()
    loss, loss_pde, loss_bc, weights = loss_fn(pinn,weights,epoch,points_generator)
    loss.backward()
    optimizer.step()
    scheduler.step()

    TOTAL_loss.append(loss.item())
    PDE_loss.append(loss_pde.item())
    BC_loss.append(loss_bc.item())

    if epoch%10 == 0:
      print(f'Epoch: {epoch} | TOTAL : {loss.item():.6f} | PDE: {loss_pde.item():.6f} | BC: {loss_bc.item():.6f}')

    if epoch%500 == 0 and epoch!=0:
        pinn.eval()
        with torch.no_grad():
            u,v,p = pinn(x_pinn.to(device),y_pinn.to(device))

            print(u.shape)
            
            U[mask_tensor] = u.squeeze().to('cpu')
            V[mask_tensor] = v.squeeze().to('cpu')
            P[mask_tensor] = p.squeeze().to('cpu')
            
            U_plot = U.reshape(n_slices,n_slices) * U_max
            V_plot = V.reshape(n_slices,n_slices) * U_max
            P_plot = P.reshape(n_slices,n_slices) * U_max**2 * rho

            plt.figure(figsize=(24,16))
            im = plt.imshow(np.sqrt(U_plot**2 + V_plot**2), origin='lower', interpolation='nearest', extent=(0,x_end,0,y_end))
            # Colorbar sotto
            cbar = plt.colorbar(im, orientation='horizontal', pad=0.1)
            cbar.set_label("Velocità")
            plt.title('U')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.gca().set_aspect((20*D)/(25*D))
            plt.tight_layout()
            plt.show()

            # 3. Save the model state dict 
            print(f"Saving model to: {MODEL_SAVE_PATH}")
            torch.save(obj=pinn.state_dict(), # only saving the state_dict() only saves the models learned parameters
                       f=MODEL_SAVE_PATH)